 # FCC - Convolutional Neural Network

**Scores**

* RMSE [2020-2050]: 1.30
* RMSE [2051-2098]: 1.51

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pylab as plt
import gc
import seaborn as sns

plt.style.use('ggplot')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname,_, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        break

import tqdm
import datetime

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
DEBUG_MODE = False

CROPS = ("maize", "wheat")# "wheat")

MODES = ('train', 'test')

FEATURES_TEMPORAL = {
    # Time series data -- 240 columns reflecting daily values for 30 days before sowing and 210 days after.
    'tas',       # Mean daily temperature
    'tasmax',    # Max daily temperature
    'tasmin',    # Min daily temperature
    'pr',        # precipitation
    'rsds'      # shortwave radiation
}

FEATURES_STATIC = {
    # Static data
    'soil_co2',  # crop, year, lon, lat, texture_class, real_year, co2, nitrogen
    # dominant USDA soil texture class (constant over time), the ambient CO2 concentration (spatially constant), the planting date and the nitrogen application rate (constant over time)
}

FEATURES = set.union(FEATURES_TEMPORAL, FEATURES_STATIC)

COLUMNS_TO_DROP = ['crop','variable']

# Sowing date
INDEX_SOW = 30  # days
# Time series data length
SEASON_LENGTH = 240  # days
# Nr. of soil texture classes
NUM_TEXTURE_CLASSES = 13  

YEAR_TRAIN_MIN = 1982
YEAR_TRAIN_MAX = 2020  # Inclusive
YEAR_TEST_MIN = 2021
YEAR_TEST_MAX = 2098

PATH_INPUT = os.path.abspath(os.path.join(os.sep, 'kaggle', 'input', 'the-future-crop-challenge'))
# PATH_INPUT = os.path.abspath(os.path.join(os.getcwd(), 'data'))  # For running the notebook locally

In [ ]:
# Reduce memory usage of a pandas DataFrame
def reduce_memory_usage(df):
    """Reduce memory usage of a pandas DataFrame."""
    # Function to iterate through columns and modify the data types
    start_mem = df.memory_usage().sum() / 1024**2
    #print(f"Memory usage of dataframe: {start_mem} MB")

    for col in df.columns:
        if col in df.index.names:  # Skip index columns, since other formats of index aren't supported by the engine
            continue

    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)  # Keep sufficient precision
            else:
                if col == "year":  # Ensure precision for grouping columns
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    #print(f"Memory usage after optimization: {end_mem} MB")
    #print(f"Decreased by {100 * (start_mem - end_mem) / start_mem}%")
    return df

In [ ]:
def load_data(crop: str, # Which crop
              mode: str, # Which dataset (i.e. train/test)
              select_only_features: bool = True, # Drop every other column (crop, year, lon, lat) if not relevant for computation
              take_subset: bool = False,  # If set to true, take a small subset of the data (for debugging purposes)
             ) -> dict:
    assert crop in CROPS
    assert mode in MODES
    
    output = dict()
    
    for f in FEATURES:
        path = os.path.join(PATH_INPUT, f'{f}_{crop}_{mode}.parquet')
        df = reduce_memory_usage(pd.read_parquet(path))

        columns_to_drop_in_df = [col for col in COLUMNS_TO_DROP if col in df.columns] 
        if columns_to_drop_in_df:
            df = df.drop(columns=columns_to_drop_in_df)

        if select_only_features:
            if f in FEATURES_TEMPORAL:  # Select only the time series data -- drop other columns
                df = df[[str(i) for i in range(SEASON_LENGTH)]]
        
        output[f] = df

        # Free up memory after processing each file
        del df  # Explicitly delete the DataFrame
        gc.collect()  # Force garbage collection
        
    if mode == 'train':
        output['target'] = pd.read_parquet(os.path.join(PATH_INPUT, f'{mode}_solutions_{crop}.parquet'))
    
    # If required, only take a subset of the data for debugging purposes -- we don't really care which samples
    if take_subset:
        num_select = 100  # Take only 100 samples from the dataset
        # Select which samples based on the index of some feature
        ixs_selected = output[tuple(FEATURES)[0]].index[:num_select]
        # Filter all dataframes
        output = {
            key: df.loc[ixs_selected] for key, df in output.items()
        }

    '''
    for df in output.values():
        df.sort_index(inplace=True)
    '''
        
    return output

In [ ]:


# Load all available data for all crops
crop_data_train = {
    crop: load_data(crop, 'train', take_subset=DEBUG_MODE, select_only_features=False) for crop in CROPS
}


crop_data_test = {
    crop: load_data(crop, 'test', take_subset=DEBUG_MODE, select_only_features=False) for crop in CROPS
}

# Separate data in features and targets (if available)
crop_features_train = {
    crop: {
        k: v for k, v in data.items() if k in FEATURES
    } for crop, data in crop_data_train.items()
}
crop_features_test = {
    crop: {
        k: v for k, v in data.items() if k in FEATURES
    } for crop, data in crop_data_test.items()
}

crop_targets_train = {
    crop: data['target'] for crop, data in crop_data_train.items()
}



In [ ]:
# crop_targets_train['wheat']
crop_features_train['maize']['tas']
# crop_features_train['wheat'].keys()

# Extra: Vernalization

In [ ]:
CROP_GDD_PARAMETERS = {
    'maize': {
        't_base': 10,  # Deg. C
        't_upper': 30  # Deg. C
    },  # Source: https://ndawn.ndsu.nodak.edu/help-corn-growing-degree-days.html
    'wheat': {
        't_base': 5.5,  # Deg. C
        't_upper': 21,  # Deg. C
    },  # Source: https://ndawn.ndsu.nodak.edu/help-wheat-growing-degree-days.html
}

# Winter wheat vernalization parameters are based on "Climate change effects on wheat phenology depends on cultivar change":
# Vernalization Module was calibrated for winter wheat in germany
# v unit is 0 for days with mean temperature below -4 deg. C or above 17 deg. C
# v unit is 1 for days with mean temperature between 4 and 10 deg. C
# v unit is linearly interpolated in missing segments
# 30 units need to be accumulated for the vernalization factor to be 1


def func_vernalization_unit(x: np.ndarray) -> np.ndarray:
    return np.interp(x, [-4, 4, 10, 17], [0, 1, 1, 0], left=0, right=0)


def func_vernalization_tres(x: np.ndarray) -> np.ndarray:
    return np.interp(x, [0, 30], [0, 1], left=0, right=1)


def compute_gdd(crop: str,
                df_tas: pd.DataFrame,
                vernalization_factor: bool = True,
                cumulative: bool = True,
               ) -> pd.DataFrame:
    t_base = CROP_GDD_PARAMETERS[crop]['t_base']
    t_upper = CROP_GDD_PARAMETERS[crop]['t_upper']
    
    df_tas = df_tas.clip(upper=t_upper, inplace=False)
    df_tas = df_tas - t_base
    df_gdd = df_tas.clip(lower=0)
    
    if vernalization_factor and crop == 'wheat':  # Account for vernalization requirement
        df_ver = func_vernalization_unit(df_tas).cumsum(axis=1)
        df_ver = func_vernalization_tres(df_ver)
        df_gdd = df_gdd * df_ver
    
    if cumulative:
        df_gdd = df_gdd.cumsum(axis=1)

    return df_gdd


# Compute the cumulative growing degree days 
for crop in CROPS:
    crop_features_train[crop]['gdd'] = compute_gdd(crop, crop_features_train[crop]['tas'], cumulative=False, vernalization_factor=False)  # Set cumulative to True?
    crop_features_test[crop]['gdd'] = compute_gdd(crop, crop_features_test[crop]['tas'], cumulative=False, vernalization_factor=False)

# Define dataset class

In [ ]:
class Dataset:
    
    fns_series = ('tasmax', 'tasmin', 'pr', 'rsds', 'gdd')
    fns_static = ('co2', 'nitrogen', 'texture_class')
    
    def __init__(self, crop: str, mode: str):
        assert crop in CROPS
        assert mode in MODES
        
        self._crop = crop
        self._mode = mode
        
        match self._mode:
            case 'train':
                self._data_series = {k: v for k, v in crop_features_train[crop].items() if k in self.fns_series}
                self._data_static = {k: crop_features_train[crop]['soil_co2'][k] for k in self.fns_static}
            case 'test':
                self._data_series = {k: v for k, v in crop_features_test[crop].items() if k in self.fns_series}
                self._data_static = {k: crop_features_test[crop]['soil_co2'][k] for k in self.fns_static}
            case _:
                raise SubmissionException(f'Unrecognized dataset mode: {mode}')

        if mode == 'train':
            self._targets = crop_targets_train[crop]
        else:
            self._targets = None
            
    def compute_norm_params(self, use_val_data: bool = False,) -> dict:
        
        if use_val_data:  # If set -> use both training and available validation data to compute mean/std
            data_series = {
                k: pd.concat([
                    crop_features_train[self._crop][k],
                    crop_features_test[self._crop][k],
                ]) for k in self.fns_series
                }
            data_static = {
                k: pd.concat([
                    crop_features_train[self._crop]['soil_co2'][k],
                    crop_features_test[self._crop]['soil_co2'][k],
                ]) for k in self.fns_static
            }
        else:
            data_series = {k: v for k, v in crop_features_train[crop].items() if k in self.fns_series}
            data_static = {k: crop_features_train[crop]['soil_co2'][k] for k in self.fns_static}            
            
        clip_std = 0.1
            
        norm_params = {
            **{
                key: (data.values.mean(), max(clip_std, data.values.std())) for key, data in data_series.items()
            },
            **{
                key: (data.values.mean(), max(clip_std, data.values.std())) for key, data in data_static.items()
            },
        }
        
        return norm_params
    
    def has_targets(self) -> bool:
        return self._targets is not None
    
    def get_index(self) -> np.ndarray:
        return self._data_static[self.fns_static[0]].index.values
    
    def _verify_data(self) -> None:
        index = self._data_static[self.fns_static[0]].index
        assert all([index.equals(df.index for df in {**self._data_static, **self._data_series}.values())])
    
    def __len__(self):
        return len(self._data_static[self.fns_static[0]])
    
    def __getitem__(self, index) -> dict:
        
        sample = {
            'series': {
                key: self._data_series[key].iloc[index].values for key in self.fns_series
            },
            'static': {
                key: self._data_static[key].iloc[index] for key in self.fns_static
            },
        }
        
        if self._targets is not None:
            sample['target'] = self._targets.iloc[index].iloc[0]
        
        return sample

# Import Pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

torch.set_default_dtype(torch.float32)

DISABLE_CUDA = False
TORCH_DEVICE = 'cuda' if torch.cuda.is_available() and not DISABLE_CUDA else 'cpu'

TORCH_DEVICE

In [ ]:
def batch_tensors(*ts):
    return torch.cat([t.unsqueeze(0) for t in ts], dim=0)


class TorchDatasetWrapper(torch.utils.data.Dataset):
    
    def __init__(self, dataset: Dataset):
        super().__init__()
        
        self._dataset = dataset
        
    def __len__(self):
        return len(self._dataset)
    
    def __getitem__(self, index) -> dict:
        
        sample = self._dataset[index]
        
        sample = self._cast_to_tensor(sample)
        
        return sample
    
    def _cast_to_tensor(self, sample) -> dict:
        
        tensor_sample = {
            'series': {
                k: torch.tensor(v, device=TORCH_DEVICE, dtype=torch.float32) for k, v in sample['series'].items()
            },
            'static': {
                k: torch.tensor(v, device=TORCH_DEVICE, dtype=torch.float32) for k, v in sample['static'].items()
            },
        }
        
        if 'texture_class' in tensor_sample['static'].keys():
            tensor_sample['static']['texture_class'] = F.one_hot(tensor_sample['static']['texture_class'].to(dtype=torch.int64) - 1, num_classes=NUM_TEXTURE_CLASSES)
            
        if 'target' in sample:
            tensor_sample['target'] = torch.tensor(sample['target'], device=TORCH_DEVICE, dtype=torch.float32)
        
        return tensor_sample
    
    def compute_norm_params(self) -> dict:
        return self._dataset.compute_norm_params()
    
    @classmethod
    def collate_fn(cls, samples: list) -> dict:
        # Define how multiple data samples should be combined to form batches
        
        data_series = {
            key: batch_tensors(*[sample['series'][key] for sample in samples]) for key in Dataset.fns_series
        }
        data_static = {
            key: batch_tensors(*[sample['static'][key] for sample in samples]) for key in Dataset.fns_static
        }
        
        batch = {
            'series': data_series,
            'static': data_static,
        }
        
        if 'target' in samples[0]:
            data_target = batch_tensors(*[sample['target'] for sample in samples])
            batch['target'] = data_target
        
        return batch

# Define CNN model

In [ ]:
class CNNModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self._norm_params = None
        
        in_channels = 5
        num_static = 2 + NUM_TEXTURE_CLASSES

        self._cnn_model = nn.Sequential(
            nn.Conv1d(in_channels=in_channels,
                      out_channels=16,
                      kernel_size=7,
                     ),
            
            nn.AvgPool1d(kernel_size=3,),
            nn.ReLU(),
            
            nn.Conv1d(in_channels=16,
                      out_channels=32,
                      kernel_size=7,
                     ),

            nn.AvgPool1d(kernel_size=3,),
            nn.ReLU(),
            
            nn.Conv1d(in_channels=32,
                      out_channels=64,
                      kernel_size=5,
                     ),
            
            nn.AvgPool1d(kernel_size=3,),
            nn.ReLU(),

            nn.Conv1d(in_channels=64,
                      out_channels=64,
                      kernel_size=3,
                     ),
            nn.ReLU(),
        )
        
        self._lin_1 = nn.Linear(256 + num_static, 64)
        self._lin_2 = nn.Linear(64, 1)
        
        
    def forward(self, xs) -> tuple:
        
        xs = self._norm_sample(xs)
        
        fns_series = ('tasmax', 'tasmin', 'pr', 'rsds', 'gdd')
        fns_static = ('co2', 'nitrogen', 'texture_class')
        
        ts = torch.cat([xs['series'][key].unsqueeze(1) for key in fns_series], dim=1)
        
        batch_size = ts.size(0)

        ts = self._cnn_model(ts)
        
        ts = torch.cat([
            ts.view(batch_size, -1),
            *{xs['static'][key].view(batch_size, -1) for key in fns_static},
        ], dim=1)
        
        ts = self._lin_1(ts)
        ts = F.relu(ts)
        ts = self._lin_2(ts)
        ts = F.relu(ts)
        
        return ts.view(-1), {}
    
    
    def _norm_sample(self, xs: dict) -> dict:
        assert self._norm_params is not None
        
        # Explicitly list features that should be normalized
        fns_series = ('tasmax', 'tasmin', 'pr', 'rsds', 'gdd')
        fns_static = ('co2', 'nitrogen')
        
        data_series = xs['series']
        data_static = xs['static']
        
        data_series_norm = dict()
        data_static_norm = dict()
        
        for key, data in data_series.items():
            if key not in fns_series:
                data_series_norm[key] = data
                continue
            mean, std = self._norm_params[key]
            data_series_norm[key] = (data - mean) / std

        for key, data in data_static.items():
            if key not in fns_static:
                data_static_norm[key] = data
                continue
            mean, std = self._norm_params[key]
            data_static_norm[key] = (data - mean) / std
        
        xs['series'] = data_series_norm
        xs['static'] = data_static_norm
                
        return xs
    
    @classmethod
    def fit(cls,
            model_name: str,
            dataset: Dataset,
            num_epochs: int = 1,
            batch_size: int = 1,
            scheduler_step_size: int = None,
            scheduler_decay: float = 0.5,
            lr: float = 1e-3,
            weight_decay: float = 1e-4,
            model_kwargs: dict = None,
            model: 'CNNModel' = None,
            print_period: int = 1,
            save_period: int = 1,
           ) -> tuple:
        
        scheduler_step_size = scheduler_step_size or num_epochs

        model_kwargs = model_kwargs or dict()
        model = (model or CNNModel(**model_kwargs)).to(TORCH_DEVICE)
        
        model._norm_params = dataset.compute_norm_params(use_val_data=False)
        
        dataset_wrapped = TorchDatasetWrapper(dataset)
        
        dataloader = torch.utils.data.DataLoader(dataset_wrapped,
                                                 batch_size=batch_size,
                                                 collate_fn=dataset_wrapped.collate_fn,
                                                 shuffle=True,
                                                )
                
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay,)
        
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                    step_size=scheduler_step_size,
                                                    gamma=scheduler_decay,
                                                    )
                
        time_start = datetime.datetime.now()

        for epoch_nr in range(1, num_epochs + 1):
            
            model.train()
            
            verbose = (epoch_nr % print_period) == 0
            save_checkpoint = (epoch_nr % save_period) == 0
            
            epoch_iter = tqdm.tqdm(dataloader, total=len(dataloader)) if verbose else dataloader
            
            losses = []
            
            for xs in epoch_iter:
                
                optimizer.zero_grad()
                                   
                loss, _, _ = model.loss(xs)
                loss.backward()
                
                optimizer.step()
                                
                losses.append(loss.item())
                loss_mean = sum(losses) / len(losses)
                
                lr = scheduler.get_last_lr()[0]
                
                if verbose:
                    epoch_iter.set_description(
                        f'{cls.__name__} training epoch [{epoch_nr:6d}/{num_epochs}] | lr: {lr:.7f} | Batch Loss: {loss.item():8.5f} | Loss Mean: {loss_mean:8.5f}',
                    )
                
                if save_checkpoint:
                    torch.save(model, f'{model_name}.pt')
                    torch.save(model.state_dict(), f'{model_name}_state.pth')
            
            scheduler.step()

        time_end = datetime.datetime.now()
        
        return model, {} 
                                   
    def loss(self, xs: dict) -> tuple:
        
        ys_true = xs['target']
        ys_pred, _ = self(xs)
        
        loss = F.mse_loss(ys_pred, ys_true)
        
        return loss, ys_pred, {}
        
    
    def predict(self,
                dataset: Dataset,
                batch_size: int = 1,
               ) -> tuple:
        
        dataset_wrapped = TorchDatasetWrapper(dataset)
        
        dataloader = torch.utils.data.DataLoader(dataset_wrapped,
                                                 batch_size=batch_size,
                                                 collate_fn=dataset_wrapped.collate_fn,
                                                 shuffle=False,
                                                )
        
        self.eval()
        
        ys_pred = []
        
        with torch.no_grad():
            for xs in tqdm.tqdm(dataloader, total=len(dataloader)):
                ys_pred_batch, _ = self(xs)
                ys_pred.append(ys_pred_batch)
        
        ys_pred = torch.cat(ys_pred, dim=0)
        
        ys_pred = ys_pred.detach().cpu().numpy()
        
        return ys_pred, pd.DataFrame({'ID': dataset.get_index(), 'yield': ys_pred}), {}

# Train the model

In [ ]:
"""main"""

dataset_maize_train = Dataset('maize', 'train')
dataset_maize_test = Dataset('maize', 'test')

FIT_MODEL = True
if FIT_MODEL:

    model_maize, _ = CNNModel.fit('model_maize',
                                  dataset_maize_train,
                                  num_epochs=20 if not DEBUG_MODE else 1,
                                  batch_size=128,
                                  scheduler_step_size=100,
                                  scheduler_decay=0.9,
                                  print_period=1,
                                  save_period=10,
                                  )

In [ ]:
_, df_maize, _ = model_maize.predict(dataset_maize_test,
                                     batch_size=1028,
                                    )
df_maize.to_csv('submission_maize2.csv', index=False, sep=',')

# %cd /kaggle/working
# from IPython.display import FileLink
# FileLink('model_state_maize.pth')

# Wheat

In [ ]:
"""main"""

dataset_wheat_train = Dataset('wheat', 'train')
dataset_wheat_test = Dataset('wheat', 'test')

FIT_MODEL = True
if FIT_MODEL:

    model_wheat, _ = CNNModel.fit('model_wheat',
                                  dataset_wheat_train,
                                  num_epochs=20 if not DEBUG_MODE else 1,
                                  batch_size=128,
                                  scheduler_step_size=100,
                                  scheduler_decay=0.9,
                                  print_period=1,
                                  save_period=10,
                                  )

    # torch.save(model_wheat, 'model_wheat.pt')
    # torch.save(model_wheat.state_dict(), 'model_state_wheat.pth')

In [ ]:
_, df_wheat, _ = model_wheat.predict(dataset_wheat_test,
                                     batch_size=1028,
                                    )
df_wheat.to_csv('submission_wheat2.csv', index=False, sep=',')

# %cd /kaggle/working
# from IPython.display import FileLink
# FileLink('model_state_wheat.pth')

# concatenate the 2 models for submissioin

In [ ]:
df2 = pd.concat([df_wheat, df_maize])

df2.to_csv('submission2.csv', index=False, sep=',')

print(df2)